# Builder Pattern Facets

Sometimes you have objects which are so complicated that you need more than one builder to create.

We can create a nice, fluent interface that allows us to jump from one builder to another.

For our example, we'll define a `Person` class with 2 different aspects: home address and employment info.

In [1]:
class Person:
    def __init__(self):
        print('Creating an instance of Person')
        # address
        self.street_address = None
        self.postcode = None
        self.city = None
        # employment info
        self.company_name = None
        self.position = None
        self.annual_income = None
        
    def __str__(self) -> str:
        return f'Address: {self.street_address}, {self.postcode}, {self.city}\n' +\
            f'Employed at {self.company_name} as a {self.postcode} earning {self.annual_income}'

We now want 2 separate builders; one for the address and another for the employment info.

In order to orchestrate these 2 separate builders, we will need a 3rd builder which will serve as a base class, and we'll use a little trick: we will initialize a blank `Person` and store it so that the subsequent builders will have an object to build up rather than creating separate objects.

We will also expose the `Person` object with a `build` method.

In [2]:
class PersonBuilder:
    def __init__(self, person=Person()):
        """Initialize a blank Person if no Person object is provided as an argument"""
        self.person = person

    def build(self):
        return self.person

Creating an instance of Person


Let's now create our subbuilders. They will inherit from our base builder.

In [3]:
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        """Instead of creating a new Person, we pass the Person we're already working on"""
        super().__init__(person)
        
    # Let's build up the Person object
    def at(self, company_name):
        self.person.company_name = company_name
        return self # fluent method
    
    def as_a(self, position):
        self.person.position = position
        return self

    def earning(self, annual_income):
        self.person.annual_income = annual_income
        return self
    
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, street_address):
        self.person.street_address = street_address
        return self

    def with_postcode(self, postcode):
        self.person.postcode = postcode
        return self

    def in_city(self, city):
        self.person.city = city
        return self

We now need to use our subbuilders from our base builder. We can create setters that call the subbuilders and pass the `self.person` object to them.

(Let's also modify the `PersonBuilder`'s `__init__()` slightly so that we can see what happens when we try to build a person without calling the subbuilders).

In [4]:
class PersonBuilder:
    def __init__(self, person=None):
        if person is None:
            self.person = Person()
        else:
            self.person = person
        
    def build(self):
        return self.person

    # Let's add the setters
    @property
    def lives(self):
        return PersonAddressBuilder(self.person)

    @property
    def works(self):
        return PersonJobBuilder(self.person)
    
# We need to copypaste the code in the previous cell block because otherwise the next cell will fail; there are no changes in the code below.
    
class PersonJobBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)
        
    def at(self, company_name):
        self.person.company_name = company_name
        return self
    
    def as_a(self, position):
        self.person.position = position
        return self

    def earning(self, annual_income):
        self.person.annual_income = annual_income
        return self
    
class PersonAddressBuilder(PersonBuilder):
    def __init__(self, person):
        super().__init__(person)

    def at(self, street_address):
        self.person.street_address = street_address
        return self

    def with_postcode(self, postcode):
        self.person.postcode = postcode
        return self

    def in_city(self, city):
        self.person.city = city
        return self

And this is how we would use our final builder in a fluent manner:

In [5]:
pb = PersonBuilder()
p = pb\
    .lives\
        .at('123 London Road')\
        .in_city('London')\
        .with_postcode('SW12BC')\
    .works\
        .at('Fabrikam')\
        .as_a('Engineer')\
        .earning(123000)\
    .build()
print(p)
person2 = PersonBuilder().build()
print(person2)

Creating an instance of Person
Address: 123 London Road, SW12BC, London
Employed at Fabrikam as a SW12BC earning 123000
Creating an instance of Person
Address: None, None, None
Employed at None as a None earning None
